In [ ]:
import re
from copy import deepcopy
import xml.etree.ElementTree as ET

#read the file
file = open('thermo_all.txt','r')

#read in each line
lines = file.readlines()

#close the file
file.close()

#dictionary
species_info = {}

#iterate over lines
for i,line in enumerate(lines):
    #spilt the line
    strings = line.split()
    
    #stop if at the end of the file
    if strings[0]=="END":
        break  
    
    #find line with species
    if strings[-1] == '1':
        #get species name
        specie = strings[0]
        
        #get the low temp min and max
        low_min = strings[-4]
        low_max = 1000.000
        
        #get the high temp min and max
        high_min = 1000.000
        high_max = strings[-3] 
        
    if strings[-1] == '2': 
        #spilt the line by number as the numbers are not broken up by spaces in txt file
        strings = re.findall(r"[+\-]?(?:0|[1-9]\d*)(?:\.\d*)?(?:[eE][+\-]?\d+)?", line)
        
        #get first 5 high coefs
        high_coeffs = []
        high_coeffs.extend(strings[0:-1])
        
        
    if strings[-1] == '3': 
        #spilt the line by number as the numbers are not broken up by spaces in txt file
        strings = re.findall(r"[+\-]?(?:0|[1-9]\d*)(?:\.\d*)?(?:[eE][+\-]?\d+)?", line)
        
        #first two are high coeefs
        high_coeffs.extend(strings[0:2])
        
        #remaining are low coeffs
        low_coeffs = []
        low_coeffs.extend(strings[2:-1])
        
    if strings[-1] == '4': 
        #spilt the line by number as the numbers are not broken up by spaces in txt file
        strings = re.findall(r"[+\-]?(?:0|[1-9]\d*)(?:\.\d*)?(?:[eE][+\-]?\d+)?", line)
        
        # get low coefs
        low_coeffs.extend(strings[0:-1])

        #Add to dictionary
        species_info[specie]={'l':{},'h':{}}
        species_info[specie]['l']['Tmax'] = low_max
        species_info[specie]['l']['Tmin'] = low_min
        species_info[specie]['l']['coeffs'] = low_coeffs
        species_info[specie]['h']['Tmax'] = high_max
        species_info[specie]['h']['Tmin'] = high_min
        species_info[specie]['h']['coeffs'] = high_coeffs

In [ ]:
#Create a xml from Dictionary
from copy import deepcopy
import xml.etree.ElementTree as ET

#Get the xml structure from file
tree = ET.parse('thermo.xml')
root = tree.getroot()

#Create species array element
speciesArray = deepcopy(root.find('phase').find('speciesArray'))

#Create string of all species
species_list = ''
for i,k in enumerate(species_info):
    species_list += ' ' + str(k)

#Add species string to copied species array element
speciesArray.text = species_list

#Delete old species array element
example_speciesArray= root.find('phase').find('speciesArray')
root.find('phase').remove(example_speciesArray)

#Append new species array eleemnt
root.find('phase').append(speciesArray)

#Make copy of a specie element to build elements off of
specie_temp = deepcopy(root.find('speciesData').find('species'))

#Delete old species entries 
example_species = root.find('speciesData').findall('species')
for specie in example_species:
    root.find('speciesData').remove(specie)

#create species from dict
for i,k in enumerate(species_info):
    
    #create element from template
    new_specie = deepcopy(specie_temp)

    #set name property to name of specie
    new_specie.set('name',str(k))
    
    #get NASA elements
    thermo = new_specie.find('thermo')
    NASA = thermo.findall('NASA')
    
    #create lower temp NASA element
    NASA[0].set("Tmax",str(species_info[k]['l']['Tmax']))
    NASA[0].set("Tmin",str(species_info[k]['l']['Tmin']))
    
    #get coeefcients string
    low_coeffs =", ".join(species_info[k]['l']['coeffs'])
    NASA[0].find('floatArray').text = low_coeffs + ", "
    
    #create higher temp NASA element
    NASA[1].set("Tmax",str(species_info[k]['h']['Tmax']))
    NASA[1].set("Tmin",str(species_info[k]['h']['Tmin']))
    
    #get coeefcients string
    high_coeffs =", ".join(species_info[k]['h']['coeffs'])
    NASA[1].find('floatArray').text = high_coeffs + ", "

    #add species to root
    root.find('speciesData').append(new_specie)
    
#create xml
tree.write('nasa.xml')